In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# to divide train and test set
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors



In [2]:
# load dataset
data = pd.read_csv('train.csv')
data.head()

C:\Users\Dns\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"cincinnati, ohio, usa",23.0
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"strafford, missouri, usa",34.0
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"st. charles county, missouri, usa",2.0
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"beaverton, oregon, usa",NaN


In [58]:
columns = ['yearOfPublication', 'publisher', 'bookAuthor', 'imageUrlS', 'imageUrlM', 'imageUrlL', 'Age']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

,userID,ISBN,bookRating,bookTitle,Location
0,276725,034545104X,0,Flesh Tones: A Novel,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,"cincinnati, ohio, usa"
2,6543,034545104X,0,Flesh Tones: A Novel,"strafford, missouri, usa"
3,8680,034545104X,5,Flesh Tones: A Novel,"st. charles county, missouri, usa"
4,10314,034545104X,9,Flesh Tones: A Novel,"beaverton, oregon, usa"


In [59]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

In [60]:
book_ratingCount = combine_book_rating.groupby(by = ['bookTitle'])['bookRating'].count().reset_index().rename(columns = {'bookRating': 'totalRatingCount'})[['bookTitle', 'totalRatingCount']]
book_ratingCount.head(10)

,bookTitle,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
5,Clifford Visita El Hospital (Clifford El Gran...,1
6,Dark Justice,1
7,Deceived,2
8,Earth Prayers From around the World: 365 Pray...,10
9,Final Fantasy Anthology: Official Strategy Gu...,4


In [67]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on='bookTitle', right_on='bookTitle', how='left')
print(rating_with_totalRatingCount.shape)
rating_with_totalRatingCount.head(10)

(1031136, 6)


,userID,ISBN,bookRating,bookTitle,Location,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,"tyler, texas, usa",60
1,2313,034545104X,5,Flesh Tones: A Novel,"cincinnati, ohio, usa",60
2,6543,034545104X,0,Flesh Tones: A Novel,"strafford, missouri, usa",60
3,8680,034545104X,5,Flesh Tones: A Novel,"st. charles county, missouri, usa",60
4,10314,034545104X,9,Flesh Tones: A Novel,"beaverton, oregon, usa",60
5,23768,034545104X,0,Flesh Tones: A Novel,"st. louis, missouri, usa",60
6,28266,034545104X,0,Flesh Tones: A Novel,"portland, oregon, usa",60
7,28523,034545104X,0,Flesh Tones: A Novel,"springfield, missouri, usa",60
8,39002,034545104X,0,Flesh Tones: A Novel,"san jose, ,",60
9,50403,034545104X,9,Flesh Tones: A Novel,"conway, arkansas, usa",60


In [66]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
print(rating_popular_book.shape)
rating_popular_book.head(10)

(288740, 6)


,userID,ISBN,bookRating,bookTitle,Location,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,"tyler, texas, usa",60
1,2313,034545104X,5,Flesh Tones: A Novel,"cincinnati, ohio, usa",60
2,6543,034545104X,0,Flesh Tones: A Novel,"strafford, missouri, usa",60
3,8680,034545104X,5,Flesh Tones: A Novel,"st. charles county, missouri, usa",60
4,10314,034545104X,9,Flesh Tones: A Novel,"beaverton, oregon, usa",60
5,23768,034545104X,0,Flesh Tones: A Novel,"st. louis, missouri, usa",60
6,28266,034545104X,0,Flesh Tones: A Novel,"portland, oregon, usa",60
7,28523,034545104X,0,Flesh Tones: A Novel,"springfield, missouri, usa",60
8,39002,034545104X,0,Flesh Tones: A Novel,"san jose, ,",60
9,50403,034545104X,9,Flesh Tones: A Novel,"conway, arkansas, usa",60


In [70]:
us_canada_user_rating  = rating_popular_book[rating_popular_book['Location'].str.contains("usa|canada")]
print(us_canada_user_rating.shape)
us_canada_user_rating .head(10)

(251615, 6)


,userID,ISBN,bookRating,bookTitle,Location,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,"tyler, texas, usa",60
1,2313,034545104X,5,Flesh Tones: A Novel,"cincinnati, ohio, usa",60
2,6543,034545104X,0,Flesh Tones: A Novel,"strafford, missouri, usa",60
3,8680,034545104X,5,Flesh Tones: A Novel,"st. charles county, missouri, usa",60
4,10314,034545104X,9,Flesh Tones: A Novel,"beaverton, oregon, usa",60
5,23768,034545104X,0,Flesh Tones: A Novel,"st. louis, missouri, usa",60
6,28266,034545104X,0,Flesh Tones: A Novel,"portland, oregon, usa",60
7,28523,034545104X,0,Flesh Tones: A Novel,"springfield, missouri, usa",60
9,50403,034545104X,9,Flesh Tones: A Novel,"conway, arkansas, usa",60
10,56157,034545104X,0,Flesh Tones: A Novel,"florissant, missouri, usa",60


In [12]:
class NullVariableTransformer(BaseEstimator, TransformerMixin):
	# Null data transformer

    def __init__(self, variables, rating_variable):
        self.variables = variables
        self.rating_variable = rating_variable
        if not isinstance(variables, list):
            raise ValueError('variables should be a list')

    def fit(self, X, y=None):
        # we need this step to fit the sklearn pipeline
        return self

    def transform(self, X):

    	# so that we do not over-write the original dataframe
        X = X.copy()
        X = X.drop_duplicates(self.variables)
        X = X.pivot(index = self.variables[0], columns=self.variables[1], values=self.rating_variable).fillna(0)
        X = csr_matrix(X.values)


In [16]:
features_to_drop = ['yearOfPublication', 'publisher', 'bookAuthor', 'imageUrlS', 'imageUrlM', 'imageUrlL']
variables = ['bookTitle','userID']
rating_variable='bookRating'
book_pipe = Pipeline([('drop_features', DropFeatures(features_to_drop=[features_to_drop])), ('NullTransformer', NullVariableTransformer(variables, rating_variable)), ('knn', NearestNeighbors(metric='cosine', algorithm='brute'))])

In [17]:
book_pipe.fit(us_canada_user_rating)

Pipeline(steps=[('NullTransformer',
                 <__main__.NullVariableTransformer object at 0x000001E5855BF460>),
                ('knn', NearestNeighbors(algorithm='brute', metric='cosine'))])

In [1]:
us_canada_user_rating.head()

NameError: name 'us_canada_user_rating' is not defined

In [18]:
query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])
distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors=6)

for i in range (0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]],  distances.flatten()[i]))

NameError: name 'us_canada_user_rating_pivot' is not defined